#### Merging propht Model  predictions with the best predictions and also with avg predictions of new city_medicines

In [97]:
import numpy as np
import pandas as pd

In [98]:
test=pd.read_csv("test_final.csv")

In [99]:
test.head()

,id,year,month,day,city,medicine,date,discounted,footfall_perday_permed,year_month,city_medicine,week,old_new
0,1,2018,7,1,1,1292,2018-07-01,0,NaN,2018_07,1_1292,26,old
1,2,2018,7,1,1,1,2018-07-01,0,NaN,2018_07,1_1,26,old
2,3,2018,7,1,1,2,2018-07-01,1,NaN,2018_07,1_2,26,old
3,4,2018,7,1,1,3,2018-07-01,0,NaN,2018_07,1_3,26,old
4,5,2018,7,1,1,4,2018-07-01,0,NaN,2018_07,1_4,26,old


In [100]:
predictions=pd.read_csv("modifiedpredxgboost_2.csv")

In [101]:
test=test[["id","date","city_medicine"]]

In [102]:
predictions.head()

,id,sales
0,1,32.854410
1,2,31.795815
2,3,113.531180
3,4,96.039190
4,5,25.185102


In [103]:
predictions=test.merge(predictions,on=["id"],how="left")

In [104]:
predictions.head()

,id,date,city_medicine,sales
0,1,2018-07-01,1_1292,32.854410
1,2,2018-07-01,1_1,31.795815
2,3,2018-07-01,1_2,113.531180
3,4,2018-07-01,1_3,96.039190
4,5,2018-07-01,1_4,25.185102


In [105]:
pr2=pd.read_csv("pred_to_testmerg.csv")

In [106]:
pr2.rename(columns={"ds":"date","yhat":"10kprpht"},inplace=True)

In [107]:
predictions=predictions.merge(pr2,how="left",on=["date","city_medicine"])

In [108]:
predictions.head()

,id,date,city_medicine,sales,10kprpht
0,1,2018-07-01,1_1292,32.854410,17.499968
1,2,2018-07-01,1_1,31.795815,NaN
2,3,2018-07-01,1_2,113.531180,59.594520
3,4,2018-07-01,1_3,96.039190,51.010142
4,5,2018-07-01,1_4,25.185102,NaN


In [109]:
predictions["10kprpht_sales"]=predictions["10kprpht"].fillna(predictions["sales"])

In [110]:
predictions.head()

,id,date,city_medicine,sales,10kprpht,10kprpht_sales
0,1,2018-07-01,1_1292,32.854410,17.499968,17.499968
1,2,2018-07-01,1_1,31.795815,NaN,31.795815
2,3,2018-07-01,1_2,113.531180,59.594520,59.594520
3,4,2018-07-01,1_3,96.039190,51.010142,51.010142
4,5,2018-07-01,1_4,25.185102,NaN,25.185102


In [111]:
preds10k=predictions[["id","10kprpht_sales"]]

In [112]:
preds10k.rename(columns={"10kprpht_sales":"sales"},inplace=True)

C:\Users\jayam\Anaconda3\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [113]:
new_citymed_df=pd.read_csv("new_citymed_avgpredictions.csv")

In [114]:
new_citymed_df=new_citymed_df[["id","sales"]]

In [115]:
new_citymed_df.dtypes

id       float64
sales    float64
dtype: object

In [116]:
new_citymed_df.rename(columns={"sales":"new_ctymed_sales"},inplace=True)

In [117]:
new_citymed_df["id"]=new_citymed_df["id"].astype("int")

In [118]:
predictions=predictions.merge(new_citymed_df,how="left",on=["id"])

In [119]:
predictions["new_ctymed_sales"]=predictions["new_ctymed_sales"].fillna(predictions["10kprpht_sales"])

In [120]:
predictions.head()

,id,date,city_medicine,sales,10kprpht,10kprpht_sales,new_ctymed_sales
0,1,2018-07-01,1_1292,32.854410,17.499968,17.499968,17.499968
1,2,2018-07-01,1_1,31.795815,NaN,31.795815,31.795815
2,3,2018-07-01,1_2,113.531180,59.594520,59.594520,59.594520
3,4,2018-07-01,1_3,96.039190,51.010142,51.010142,51.010142
4,5,2018-07-01,1_4,25.185102,NaN,25.185102,25.185102


In [121]:
newmed_10kpr=predictions[["id","new_ctymed_sales"]]

In [122]:
newmed_10kpr.head()

,id,new_ctymed_sales
0,1,17.499968
1,2,31.795815
2,3,59.594520
3,4,51.010142
4,5,25.185102


In [123]:
newmed_10kpr.rename(columns={"new_ctymed_sales":"sales"},inplace=True)

C:\Users\jayam\Anaconda3\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [124]:
newmed_10kpr.head()

,id,sales
0,1,17.499968
1,2,31.795815
2,3,59.594520
3,4,51.010142
4,5,25.185102


In [46]:
newmed_10kpr.to_csv("newmedprphtsales.csv",index=False)